Import libaries

In [1]:
import numpy as np
import pandas as pd
import pvlib
import importlib
import datetime
import pytz

Local libary

In [40]:
import av_utils as av
importlib.reload(av)

<module 'av_utils' from 'c:\\Users\\david.jung\\apv-app\\app-apv\\av_utils.py'>

Functions with inputs and results

In [5]:
# Location for TMY download
latitude = -15.087836  # latitude of the location (Mocambinho) -> [INPUT]
longitude = -44.015762 # [INPUT]
#altitude  = 454
tz= 'Brazil/East'  #[INPUT]

# Dowload of TMY from PVGIS, altitude 
tmy, altitude = av.tmy_download(latitude, longitude, tz)

Download of TMY data: complete


Climate Analisis (new functions in av_utils)

In [44]:
#create daily dataframe
df_d = av.daily_climate_data(tmy)
df_d.head(2)

,rs_kW_m2,rs_MJ_m2,tmean_C,tmax_C,tmin_C,rh_%,rhmax_%,rhmin_%,wind_speed_m_s,wind_speedmax_m_s
time,,,,,,,,,,
2022-01-01 00:00:00-03:00,8.091,29.1276,30.616250,35.96,25.04,41.902917,70.4,22.45,0.806667,1.72
2022-01-02 00:00:00-03:00,7.675,27.6300,30.488333,34.95,25.93,46.108333,62.1,34.20,1.402917,2.28


In [45]:
#plot selected data (ver columnas de df_d)
av.plot_climate_data(df_d,selected_data1 = 'rs_kW_m2', selected_data2 = 'wind_speed_m_s' )

In [41]:
#calculate evapotranspiration with two methods
df_pet = av.calc_evapo(df_d,lat= latitude, alt=altitude, method1="FAO-56",method2="Hargreaves")

The values for total anual Evapotranspiration [mm/m2]are
Penman               1947.57
FAO-56               1723.42
Priestley-Taylor     1660.15
Kimberly-Penman      1483.82
Thom-Oliver          1699.74
Blaney-Criddle       1341.79
Hamon                1270.99
Romanenko            3129.86
Linacre              1588.54
Haude                1726.37
Turc                 1692.16
Jensen-Haise         2341.21
Mcguinness-Bordne    2380.99
Hargreaves           1118.29
FAO-24               1687.44
Abtew                1705.55
Makkink              1583.16
Oudin                1619.72
dtype: float64


In [46]:
#create climate summary
av.climate_summary(df_d, df_pet)

                   yearly total  daily average max month  max month value  \
parameter                                                                   
rs_kW_m2                2180.54           5.97       Oct             7.20   
rs_MJ_m2                7849.93          21.51       Oct            25.92   
tmean_C                 9408.19          25.78       Oct            28.64   
tmax_C                 11205.30          30.70       Oct            34.64   
tmin_C                  7694.50          21.08       Nov            23.56   
rh_%                   20556.25          56.32       Ene            79.86   
rhmax_%                27833.02          76.25       Ene            92.11   
rhmin_%                14095.73          38.62       Ene            64.36   
wind_speed_m_s           660.81           1.81       Oct             2.44   
wind_speedmax_m_s        991.67           2.72       Oct             3.33   
FAO-56                  1723.42           4.72       Oct             6.57   

PV Simulation

In [7]:
# Inputs for PV simulation
track = True # [INPUT]
pvrow_azimuth = 90 #[INPUT]
pvrow_tilt = 60 #[INPUT]

# fijos
albedo = 0.2
n_pvrows = 3
pvrow_width = 1
pvrow_pitch = 4
pvrow_height = 3
bifaciality = 0.9

# PV simulation (pvlib viewfactors)
pv = av.pv_yield(tmy_data = tmy, 
                albedo = albedo, 
                track = track, 
                pvrow_azimuth = pvrow_azimuth, 
                pvrow_tilt = pvrow_tilt, 
                n_pvrows = n_pvrows, 
                pvrow_width = pvrow_width, 
                pvrow_pitch = pvrow_pitch, 
                pvrow_height = pvrow_height, 
                bifaciality = bifaciality)
print("PV generation is "+str(pv.sum()/1000)+" kWh/kWp/year")

PV generation is 2158.15807378367 kWh/kWp/year


In [8]:
# Input for LCOE calculation
kWp = 1             # [INPUT]
capex = 1000        # USD per kWp   # [INPUT] 
opex = 20             # USD per kWp/y # [INPUT]
wacc = 0.055            # [INPUT]
degre = 0.005           # [INPUT]
inflation = 0.03        # [INPUT]
N = 25                  # [INPUT]

# LCOE calculation (can be externalized as functions in av_utils)
cashflow= pd.DataFrame(index=range(0,N))
cashflow["OPEX_des"] = (opex * kWp * (1+inflation)**cashflow.index) / (1+wacc)**cashflow.index
cashflow["EG_des"] = (pv.sum()/1000 * kWp * (1-degre)**cashflow.index) / (1+wacc)**cashflow.index
LCOE =  (capex * kWp + cashflow["OPEX_des"].sum() ) / cashflow["EG_des"].sum()
print("LCOE of the simulated system is "+str(round(LCOE,2))+" USD/kWh")

LCOE of the simulated system is 0.05 USD/kWh


Next steps:
- create website where inputs can be changed and results displayed (https://streamlit.io/, https://www.youtube.com/watch?v=VqgUkExPvLY) 
- Add other APV dimensions: Water costs and water protection...
